Beste Peter, enorm bedankt voor je hulp met mijn eerste Python code! Ik hoop dat het een beetje helder is wat ik overal probeerde te doen, en tot vanmiddag :)

In onderstaande wilde ik graag het woordaantal van het corpus berekenen.



In [28]:
from collections import Counter 
import os 
import re
from nltk import word_tokenize , sent_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import nltk
import math
import pandas as pd

def remove_punctuation(words):
    new_list= []
    for w in words:
        if w.isalnum():
            new_list.append( w )
    return new_list

def remove_extension(file):
    return( file[:file.rindex('.'):] )


lemmatiser = WordNetLemmatizer()
def lemmatise(full_text):
    sentences = sent_tokenize(full_text)
    lemmatised_text = ''

    for sent in sentences:
        words = word_tokenize(sent)
        words = remove_punctuation(words)
        pos = nltk.pos_tag(words)
        lemmatised_sent = ''

        for i,word in enumerate(words):
            word = word.lower()
            posTag = ptb_to_wordnet( pos[i][1] )

            if re.search( r'\w+' , posTag , re.IGNORECASE ):
                lemma = lemmatiser.lemmatize( words[i] , posTag )
                lemmatised_sent += lemma + ' '
            else:
                lemmatised_sent += word + ' '
        lemmatised_text += lemmatised_sent

    return lemmatised_text

from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords.append('illustration') 
stopwords.append('chapter')

def ptb_to_wordnet(PTT):

    if PTT.startswith('J'):
        ## Adjective
        return 'a'
    elif PTT.startswith('V'):
        ## Verb
        return 'v'
    elif PTT.startswith('N'):
        ## Noune
        return 'n'
    elif PTT.startswith('R'):
        ## Adverb
        return 'r'
    else:
        return ''


dir = 'Lemmatised'
if not os.path.isdir(dir):
    os.mkdir(dir)

> De onderstaande cel lemmatiseert alle teksten en berekent de woordfrequenties. De boeken moeten worden opgeslagen in de map 'Corpus'

In [2]:
dir = 'Corpus'


texts = []
metadata = dict()

freq = Counter()

for file in os.listdir(dir):
    if re.search('txt$',file):
        print(f'{file} ...')
        year = file[file.rindex('_')+1:].replace('.txt','')
        if re.search(r'\-' , year):
            year = year[ : year.index('-')]
        year = int(year)        
        metadata[file] = year
        file_handler = open(os.path.join(dir,file),encoding='utf-8')
        full_text = file_handler.read()  
        
        lemmatised = lemmatise(full_text)
        out_file = os.path.join( 'Lemmatised' , f'{remove_extension(file)}_lemma.txt')
        texts.append(out_file)
        with open( out_file ,'w',encoding='utf-8') as out:
            out.write(lemmatised)
        
        words = word_tokenize(full_text.lower())
        words = remove_punctuation(words)
        words = [word for word in words if word not in stopwords]
        freq.update(words)
print('Done!')

Lang_BooksandBookmen_1887.txt ...
Dibdin_Bibliomania_1876.txt ...
Lang_TheLibrary_1881.txt ...
Nodier_TheBookCollector_1841-1951.txt ...
Harper_BookLoversBibliomaniacsandBookClubs_1904.txt ...
Slater_RomanceofBook-Collecting_1898.txt ...
Blades_EnemiesofBooks_1888.txt ...
Vincent_BibliotaphAndOtherPeople_1899.txt ...
Slater_Book CollectingAGuideforAmateurs_1892.txt ...
Burton_TheBookHunter_1862.txt ...
Hazlitt_TheBookCollector_1904.txt ...
Browne_IntheTrackoftheBookWorm_1897.txt ...
Wheatley_BookLoversLibraryHowtoFormaLibrary_1886.txt ...
Mermet_CrazyBookCollecting_1894.txt ...
Allan_theBookHunteratHome_1922.txt ...
Humphreys_thePrivateLibrary_1800.txt ...
Dibdin_LibraryCompanion_1824.txt ...
Elton_TheGreatBookCollectors_1893.txt ...
Koopman_BookLoverandhisBooks_1917.txt ...
Merryweather_BibliomaniaintheMiddleAges_1900.txt ...
Dibdin_BibliographicalDecameron_1817.txt ...
Maclaren_BooksandBookmen_1912.txt ...
Arnod_FirstReportofaBookCollector_1898.txt ...
DeBury_Philobiblon_1996.txt ...

In [3]:
tokens = sum(freq.values())
types = len(freq.keys())
print(f'Number of tokens: {tokens}')
print(f'Number of types: {types}')

Number of tokens: 1208183
Number of types: 78626


In onderstaande zou ik graag de 100 meest frequente woorden zien. 

In [4]:
for word,count in freq.most_common(500): 
    print( f'{word} => {count}') 

books => 9741
one => 8093
book => 5248
may => 5085
library => 4828
upon => 4668
first => 4564
de => 4425
copy => 3991
would => 3877
great => 3510
many => 3493
printed => 3300
work => 3114
old => 3112
time => 3062
two => 3022
volumes => 3008
edition => 2926
volume => 2680
much => 2470
years => 2365
works => 2346
man => 2331
well => 2316
good => 2280
little => 2237
must => 2234
collection => 2183
day => 2150
us => 2132
every => 2064
yet => 2055
made => 2028
paper => 1976
history => 1973
even => 1953
copies => 1943
english => 1938
see => 1934
could => 1910
said => 1895
also => 1874
like => 1869
vol => 1842
large => 1822
published => 1814
john => 1741
without => 1738
et => 1732
found => 1695
folio => 1685
part => 1683
never => 1683
among => 1673
life => 1656
sale => 1652
however => 1646
sir => 1636
might => 1632
catalogue => 1596
author => 1533
london => 1485
0 => 1478
last => 1474
new => 1470
another => 1463
year => 1448
three => 1430
let => 1422
rare => 1410
long => 1396
8vo => 1367
cent

> Frequenties van de gelemmatiseerde woorden

In [5]:
freq_lemma = Counter()
dir = 'Lemmatised'
for file in os.listdir(dir):
    path = os.path.join(dir,file)
    with open(path,encoding='utf-8') as fh:
        full_text = fh.read()
        words = word_tokenize(full_text.lower())
        words = [word for word in words if word not in stopwords]
        freq_lemma.update(words)

In [6]:
tokens = sum(freq_lemma.values())
types = len(freq_lemma.keys())
print(f'Number of tokens: {tokens}')
print(f'Number of types: {types}')

Number of tokens: 1215639
Number of types: 73025


In [7]:
for word,count in freq_lemma.most_common(500): 
    print( f'{word} => {count}') 

book => 13883
one => 8293
copy => 5962
library => 5730
volume => 5593
work => 5429
may => 5085
de => 4825
upon => 4668
first => 4565
great => 4328
say => 4309
edition => 4114
make => 3974
would => 3877
time => 3806
year => 3767
many => 3493
print => 3447
old => 3313
find => 3156
see => 3042
two => 3022
day => 3016
know => 2941
give => 2893
good => 2875
take => 2734
collection => 2602
well => 2567
much => 2470
man => 2334
little => 2237
must => 2234
part => 2228
write => 2209
paper => 2170
come => 2167
us => 2162
catalogue => 2114
large => 2095
author => 2080
every => 2064
history => 2063
page => 2056
yet => 2055
even => 1989
name => 1988
early => 1979
like => 1959
english => 1938
publish => 1917
could => 1910
place => 1899
folio => 1883
also => 1874
vol => 1842
sale => 1839
use => 1810
life => 1796
price => 1789
go => 1782
present => 1762
letter => 1741
john => 1741
without => 1738
read => 1737
et => 1732
call => 1722
never => 1683
seem => 1673
among => 1673
however => 1646
think => 16

## Lexicon

In onderstaande zou ik graag mijn lexicons inladen, maar ik begrijp de code niet 100 procent dus heb het er graag over of het zo goed zit, en hoe ik eventueel makkelijker alle lexicons tegelijk kan inladen. Ook lijkt het me mooi het grafisch te kunnen vormgeven, maar ik wist even niet hoe ik dat goed moest ombouwen. Ook zou ik graag eerst het corpus lemmatisen, maar ik wist niet zo goed hoe ik dat aan moest pakken. 

In [8]:
dir = 'Lexicons'
lexicons = dict()

for file in os.listdir(dir):

    topic = file[ : file.rindex('.') ]
    words = []
    
    with open( os.path.join(dir,file) , encoding = 'utf-8' ) as file_handler:   
        for l in file_handler: 
            if re.search( r'\w' , l ):
                words.append(l.strip())

    lexicons[topic] = words    

In [9]:
print(f'{len(lexicons)} lexicons were found.')

4 lexicons were found.


In [10]:

dir = 'Corpus'

csv = open( 'lexicon.csv' , 'w' , encoding = 'utf-8' )

## print header
csv.write( 'title' )
for l in lexicons:
    csv.write( f',{l.lower().strip() }' )
csv.write('\n')

dir = 'Lemmatised'
for file in os.listdir( dir ):
    if re.search( r'\.txt$' , file ):
        text_title = remove_extension( file )
        text_title = re.sub( r'_\d{4}_lemma' , '' , text_title)
        
        csv.write(text_title)
        path = os.path.join( dir, file )
        with open(path,encoding='utf-8') as fh:
            full_text = fh.read()
        
        print( 'Performing semantic tagging for {} ...'.format( file ) )
        
        words = word_tokenize(full_text)
        words = remove_punctuation(words)
        tokens = len(words)
        
        for l in lexicons:            
            count_occurrences = 0
            for word in l:
                count_occurrences += 1
            
            csv.write( ',{}'.format( count_occurrences / tokens ) )
        csv.write('\n')
        
csv.close()
print("Done!")


Performing semantic tagging for Browne_IntheTrackoftheBookWorm_1897_lemma.txt ...
Performing semantic tagging for Wheatley_BookLoversLibraryHowtoFormaLibrary_1886_lemma.txt ...
Performing semantic tagging for Lang_BooksandBookmen_1887_lemma.txt ...
Performing semantic tagging for Arnod_FirstReportofaBookCollector_1898_lemma.txt ...
Performing semantic tagging for Dibdin_LibraryCompanion_1824_lemma.txt ...
Performing semantic tagging for Nodier_TheBibliomaniac_1894_lemma.txt ...
Performing semantic tagging for Slater_HowtoCollectBooks_1905_lemma.txt ...
Performing semantic tagging for Slater_RomanceofBook-Collecting_1898_lemma.txt ...
Performing semantic tagging for deMermet_CrazyBookCollecting_1894_lemma.txt ...
Performing semantic tagging for Beresford_Bibliosophia or Book-Wisdom_1810_lemma.txt ...
Performing semantic tagging for Mermet_CrazyBookCollecting_1894_lemma.txt ...
Performing semantic tagging for DeBury_Philobiblon_1996_lemma.txt ...
Performing semantic tagging for Maclaren_

In [11]:
df = pd.read_csv('lexicon.csv')

In [12]:
for i,row in df.iterrows():
    print(f'{row["title"]}')
    for lexicon in lexicons.keys():
        print(f"{lexicon}: {round(row[lexicon],5)*100}% of total number of tokens")

Browne_IntheTrackoftheBookWorm
love: 0.016% of total number of tokens
magic: 0.02% of total number of tokens
hunt: 0.016% of total number of tokens
madness: 0.027999999999999997% of total number of tokens
Wheatley_BookLoversLibraryHowtoFormaLibrary
love: 0.01% of total number of tokens
magic: 0.012% of total number of tokens
hunt: 0.01% of total number of tokens
madness: 0.017% of total number of tokens
Lang_BooksandBookmen
love: 0.012% of total number of tokens
magic: 0.015% of total number of tokens
hunt: 0.012% of total number of tokens
madness: 0.022000000000000002% of total number of tokens
Arnod_FirstReportofaBookCollector
love: 0.027% of total number of tokens
magic: 0.034% of total number of tokens
hunt: 0.027% of total number of tokens
madness: 0.048% of total number of tokens
Dibdin_LibraryCompanion
love: 0.002% of total number of tokens
magic: 0.003% of total number of tokens
hunt: 0.002% of total number of tokens
madness: 0.004% of total number of tokens
Nodier_TheBiblioman

Hieronder heb ik alle code herhaald voor de andere 3 corpora. Waarschijnlijk voldoet dat niet aan de python-waarden om efficient te zijn maar het was even het enige wat ik kon bedenken :)

In [13]:
def make_regex(lexicon_list):
    regex = ''
    for i,term in enumerate(lexicon_list):
        regex += f'(\\b{term}\\b)'
        if i < len(lexicon_list)-1:
            regex += '|'
    return r'{}'.format(regex)



def concordance_word( text, regex , width = 10 ):

    concordance = []
    distance = math.floor( width /2 )

    segment_length = 0

    words = word_tokenize( text )
    words = remove_punctuation( words )
    i = 0
    for w in words:
        if re.search( regex , w , re.IGNORECASE ):
            match = ''
            for x in range( i - distance , ( i + distance ) + 1 ):
                if x >= 0 and x < len(words):
                    if len(words[x]) >= 0:
                        if re.search( regex , words[x] , re.IGNORECASE ):
                            match += words[x].upper() + ' '
                        else:
                            match += words[x] + ' '
            concordance.append( match )

        i += 1

    return concordance


        

> Hieronder wordt voor ieder lexicon bekeken hoe vaak de woorden voorkomen. De berekeningen zijn percentages van het totaal aantal woorden. 

In [14]:

for text in texts:
    print(text)
    for lexicon in lexicons:
        regex = make_regex(lexicons[lexicon])
        full_text = open(text,encoding='utf-8').read()
        c = concordance_word( full_text, regex , width = 20 )
        if len(c)>0:
            print('\n'+lexicon.upper(),end='\n\n')
            for line in c:
                print(line,end='\n')
        



Lemmatised/Lang_BooksandBookmen_1887_lemma.txt

LOVE

sound criticism of the mania Already in the seventeenth century LOVER of the tiny Elzevirian book wax pathetic over date already 
and that the paper type ornament and be their main ATTRACTION to these we must now add the rarity of really 
Elzevir the late Hepworth Dixon introduce into one of his ROMANCE a ROMANTIC Elzevir Greek Testament worth its weight in Casual 
late Hepworth Dixon introduce into one of his ROMANCE a ROMANTIC Elzevir Greek Testament worth its weight in Casual remark of 
be pearl of considerable price when a man be first SMITTEN with the pleasant fever of it be for Elzevirs that 
the King of Lilliput be taller than any of his COURT by almost the breadth of a nail and that his 
reprint of any modern volume on cooking the Pastissier be CHERISH because it be so very rare the tract pass into 
1821 Brunet say with a feigned brutality for he dearly LOVE an Elzevir Till now i have disdain to admit this 
piece of luck at 


MAGIC

more of old talk about book and bookseller than about FAIRY and the Entretiens be publish in 1699 about sixteen year 
of cook be detrimental to literature Just as nursery book FAIRY tale and the like be destroy from generation to generation 
be already make 12 Picture Elzevir Sage Not for these CHARM but for its rarity be the Pastissier covet in an 
by Rare Ben Jonson in Hoxton Fields the burning of WITCH be naturally not an uncommon item in parish register and 
and one man be execute for the imaginary crime of WITCHCRAFT a grave for a WITCH sixpence be an item in 
for the imaginary crime of WITCHCRAFT a grave for a WITCH sixpence be an item in the municipal account and the 
tender mercy of a Scotch Cetewayo s who smelt out WITCH be only some two century in the rear of our 
Cetewayo and Saul as to the wickedness of suffer a WITCH to live As late as 1691 the register of Holy 
1691 the register of Holy Island Northumberland mention William Cleugh BEWITCH to death and the SUPERSTI


LOVE

GEYLER Navis Stultifera sign iiij rev BIBLIOMANIA OR A BIBLIOGRAPHICAL ROMANCE ILLUSTRATED WITH CUTS BY THOMAS FROGNALL DIBDIN and improved Edition 
in all probability have attain a considerable reputation how he LOVE to embellish almost to satiety a favourite work may be 
affliction and if the voice of those whom we tenderly LOVE whether parent child be hear from grave trust it would 
the sweet specimen of modern biography have sink into every HEART and receive an eulogy from every tongue yet be his 
employ so credulous a bibliographical forager a modern collector and LOVER of copy will witness with shudder among Bagford immense collection 
period of human existence the emotion of friendship or of LOVE be weaken or subdue as old age advance but the 
the variety of his erudition and the intenseness of his ARDOUR in 18 i discover no other notorious example of the 
Petrarch of him and in order to excite the same ARDOUR in his countryman or rather to propagate the disease of 
toil 


MAGIC

the East seem not to have be insensible to the CHARM of Falconry if we be to judge from the evidence 
be among the first who teach their countryman to be CHARM with the elegance and copiousness of their own language how 
realm be not all clear of this detestable fact but CURSE be that belly which seeketh to be feed with such 
forty article or about 560 volume 6 Natural History Astrology SORCERY Probably not more than 100 volume 7 and this comprehend 
margin on a rare and lovely print introduce and be CHARM with the soft and coax manner in which by the 
have be cover by the green grass turf the gentle ELF of take care to weave a chaplet to hang upon 
upon the rarity of which bibliographer delight to expatiate his OCCULT PHILOSOPHY accord to Bayle in 1531 at least the Elector 
precious volume of collector which he have see like Macbeth WITCH Come like shadow so depart Footnote 186 Tenni cultu victuque 
ready knack which he hath acquire by mean of some MAGICAL art in his foresaid 


HUNT

and of hope that the true object of and literary PURSUIT might have be fully and fairly develop the perusal of 
which grow out of an excessive attachment to any particular PURSUIT be that PURSUIT horse 5 hawks DOG GUN snuff box 
of an excessive attachment to any particular PURSUIT be that PURSUIT horse 5 hawks DOG GUN snuff box 6 old china 
to any particular PURSUIT be that PURSUIT horse 5 hawks DOG GUN snuff box 6 old china coin or rusty armour 
any particular PURSUIT be that PURSUIT horse 5 hawks DOG GUN snuff box 6 old china coin or rusty armour may 
ascertain the general usage of the French language with our HUNTSMAN in the 15th century Beasts of Venery where so ever 
be the another be the the and the and the WILD and ye shall see my dear child other beastes all 
seek the fyrst worde to the ho dis that the HUNTER shall out pit Is at the kenell doore whan he 
at this present eventful moment afflict not only with the DOG 16 but the BOOK disease Fire in each eye and 
breech was

...


MADNESS

TINSLEY 12 PAGE OF ORIGINAL OF HARDY FAR FROM THE MADDING CROWD 14 BERNARD QUARITCH 14 TITLE OF OF LYFORD REDIVIVUS 
out of the window the joy of rug be a DELUSION and a snare they can not be pick up here 
since quite unexpectedly the original draft of Far from the MADDING Crowd turn up in London its author when informed of 
Illustration FACSIMILE of A PAGE OF HARDY FAR FROM THE MADDING CROWD MUCH REDUCED IN SIZE Illustration BERNARD QUARITCH the extensive 
book begin and will end with Boswell of Johnson Thou FOOL to seek companion in a crowd Into thy room and 
he finally reach the conclusion that Boswell be a great FOOL he be a very great writer Absurdity can go no 
so easy to keep ring the change on Macaulay any FOOL with a pen can do it some time ago apropos 
and at him sometimes do not think him altogether a FOOL he be vain and foolish yes and inquisitive yet while 
early habit acquire that i should expose myself yet the DELUSION of Westminster Hall of brilliant reputatio


MAGIC

for here shall morn her choice gifts renew and nightly FAIRY round his grave shall dance Ah why delay Begin ye 
lecture a covered playground account the Spectator and dance as MAGICAL word hear for the first time i rub my hand 
light the fire the designer and the engraver of these CHARM little specimen of their early effort in art have recently 
to whom the sister Graces lend their aid and rural ELF fantastic homage pay for whom the Naïads leave their bed 
approach to awe and dread this place have an indescribably MAGIC CHARM for me and when a i be constantly stroll 
to awe and dread this place have an indescribably MAGIC CHARM for me and when a i be constantly stroll thither 
very cold or unimpassioned strain when that his voice do CHARM the attentive throng and every ear be link unto his 
for though sometimes it make thee weep and wail and CURSE thy star and early drudge and lateWithouten that would come 
CHOICE of A PROFESSION and his bright blue penetrate eye CHARM with the


MADNESS

a 320021 4800 円 Dilsin DIBDIN BIBLIOMANIA or A BIBLIOGRAPHICAL ROMANCE A NEW EDITION WITH CONSIDERABLE ADDITIONS 
work must soon again be number among rare book the BIBLIOMANIA be write in dialogue or conversation and the character duced 
it will be always sulted as a first authority the BIBLIOMANIA be evidently form in the first instance on the plan 
of note for Niebührh read Niebühr 6 of text for LOCO read foco 3 of note for through read throughout 18 
rapidly decline before he leave the country a complication of DISORDER with a spirit bring on an inflammation of the bowel 
of nature or consumption as he have labour under many DISORDER a long time to describe what i have undergone and 
catch or fancy i catch the electric spark of the BIBLIOMANIA my master be now and then the purchaser of old 
all the wild principle of the French revolution in its MAD day go over to America i know not upon what 
DAYS 65 Merry Wives of Windsor excite me almost to DELIRIUM oh yet more fatal re


MAGIC

reasoning or argument will deter the real booklover from his CHARM pursuit the love of book and their possession be to 
of the binder and if to these be add the CHARM of a provenance or a dedication or a fine you 
within the domain of collectorship First edition have an irresistible CHARM for the PREFACE ix lover a first edition bring us 
subdue the soul or of captivate the heart by the MAGIC of eloquence or of poetry these devote themselves to the 
be the lack of this one object may destroy the CHARM of all that he have already win it be intolerable 
no passion so dominant as never to yield to the CHARM of it be easy to discover fresh abuse which may 
in the marbling of the edge Books in these style CHARM the eye please the taste and be the delight of 
impetuous or insinuating burst of eloquence more move by the CHARM of poetry because i find them in volume glitter with 
of exquisite vas and bronze to give it a crown CHARM Seneca declaim against this in the lover of book he 
h


MAGIC

of these write conversation but when the voice cease the CHARM vanishes it be certain that the reader of be somewhat 
other day which Nodier evoke let us look at the CHARM type of a who be worthy of the name Silvestre 
say in one of those confidence in which lie the CHARM of his criticism if i become blind i think i 
you have the curiosity to learn the identity of this CHARM man with the single unpardonable fault it can be find 
a book but eager to know its content these two CHARM mind have create laughter more than once when in answer 
date and the printer s name the consider him a SORCERER but about this there be a difference of opinion Illustration 
modest 5 of Horace or the elastic cover of the FAIRY tent which if necessary could shield the whole army of 
its very self we have a jolly dinner that day CHARM woman fresh oyster intelligent people and champagne i reach the 

HUNT

have be themselves the gainer thereby the belongs to a RACE of refined egotist he know and taste t

In [27]:
dir = 'Corpus'
for text in os.listdir(dir):

    if re.search( r'txt$' , text):
        print(text)
        for lexicon in lexicons:
            print(lexicon, end='\n\n')
            regex = make_regex(lexicons[lexicon])
            path = os.path.join(dir,text)
            full_text = open(path,encoding='utf-8').read()
            full_text = re.sub( r'\s+' , ' ' , full_text )
            sentences = sent_tokenize(full_text)
            for s in sentences:
                if re.search(regex,s,re.IGNORECASE):
                    print(s)

    

Lang_BooksandBookmen_1887.txt
love

You read in novels about “priceless little Elzevirs,” about books “as rare as an old Elzevir.” I have met, in the works of a lady novelist (but not elsewhere), with an Elzevir ‘Theocritus.’ The late Mr. Hepworth Dixon introduced into one of his romances a romantic Elzevir Greek Testament, “worth its weight in gold.” Casual remarks of this kind encourage a popular delusion that all Elzevirs are pearls of considerable price.
When a man is first smitten with the pleasant fever of book-collecting, it is for Elzevirs that he searches.
Dr. Lemuel Gulliver remarks that the King of Lilliput was taller than any of his court by almost the breadth of a nail, and that his altitude filled the minds of all with awe.
‘You can have it for six sous,’ he answered; ‘is it too much?’ ‘No,’ said I, and, trembling a little, I handed him the thirty centimes he asked for the ‘Pastissier François.’ You may believe, my friend, that after such a piece of luck at the start, one

He had the ‘Ode to Despair’ of Smith (now a comic writer), and the ‘Love Lyrics’ of Brown, who is now a permanent under-secretary, than which nothing can be less gay nor more permanent.
On the day of which I speak he had secured a volume of love-poems which the author had done his best to destroy, and he had gone to his club and read all the funniest passages aloud to friends of the author, who was on the club committee.
Paying the sum demanded, and trailing his vast load of German romance, poor Blinton followed the fiend.
‘Tirante il Bianco Valorosissimo Cavaliere’ was being competed for, an excessively rare romance of chivalry, in magnificent red Venetian morocco, from Canevari’s library.
His copy (almost unique) of Richard Barnfield’s much too ‘Affectionate Shepheard’ was coupled with odd volumes of ‘Chips from a German Workshop’ and a cheap, imperfect example of ‘Tom Brown’s School-Days.’ Hookes’s ‘Amanda’ was at the bottom of a lot of American devotional works, where it kept compa

If we are to understand the book-hunter, we must never forget that to him books are, in the first place, _relics_.
In 1664, Naudé, who was a learned and ingenious writer, the apologist for “great men suspected of magic,” published the second edition of his ‘Avis pour dresser une Bibliothèque,’ and proved himself to be a true lover of the chase, a mighty hunter (of books) before the Lord.
The _bouquin_ is the “small, rare volume, black with tarnished gold,” which lies among the wares of the stall-keeper, patient in rain and dust, till the hunter comes who can appreciate the quarry.
He might have been tracked by his trail of romances, as was Hop-o’-my-Thumb, in the fairy tale, by the white stones he dropped behind him.
He _invented_ an edition, and put the collector on the scent, which he followed vainly, till he died of the sickness of hope deferred.
We side with the poor hunters of the wild game, who hang over the fourpenny stalls on the _quais_, and dive into the dusty boxes after lit

The former, on his death-bed, wrote Collins a letter to be delivered to him, after his decease, which was full of affection and good advice.]
A mortifying Fast Diet at Court.
Those who recollect the zeal and scholarship of this great book-collector, and the precious gems with which his library[46] was stored from the cabinets of De Boze and Gaignat, as well as of Mead and Folkes, cannot but sigh with grief of heart on the thought of such a victim!
It appeared; and even Askew, with all his skill in medicine and books, fell lifeless before it--bewailed, as he was beloved and respected!
Of CONSUL SMITH I am unable to present the lover of VIRTU with any particulars more acceptable than the following.
_English Poetry, Romance, and Miscellanies._ This interesting part of the collection comprehended about 355 articles, or probably about 750 volumes: and if the singularly rare and curious books which may be found _under these heads alone_ were now concentrated in one library, the owner of them

Morhof died A.D. 1691, in his 53rd year: beloved by all who knew the excellent and amiable qualities of his head and heart.
My invariable love of truth impels me to state facts as they arise.
Yonder sharp and well turned countenances, at the entrance of the grotto, are fixed there as representations of CARDINAL QUIRINI[139] and GOUJET; the _Bibliothéque Françoise_ of the latter of whom--with which I could wish book collectors, in general, to have a more intimate acquaintance--has obtained universal reputation.
It is a little surprising that Lysander, in his love of grand national biographical works, mingled with bibliographical notices, should have omitted to mention the _Bibliotheca Lusitana_ of Joaov and Barbosa, published at Lisbon, 1741, in four magnificent folio volumes.
A lover of Portuguese literature will always consider this as "opus splendidissimum et utilissimum."]
From my heart I pity and forgive you.
But only look upon the bust of DE BURE; and every time that you open his 

...

Let me not, however, quit the sanctum sanctorum of my indulgent master, without just mentioning that, it was here, for the first time, I caught, or fancied I caught, the electric spark of the BIBLIOMANIA.
65 Merry Wives of Windsor, excited me almost to delirium : —and oh !
A simple but severe incident entirely cured me of this mania ; but my love of art was, if possible, increasing every day.
Two SUCH volumes cannot fail, in every vicissitude of the BIBLIOMAnia, to hold their heads as high as ever.
Two works, at this * The expression is in Richard De Bury's Philobiblon ; a work, about which there is mention made, almost to raving, in the Bibliomania, p. 245-7 ; Bibl.
Mad, indeed, we were, and desired so to be called-if an occasional deviaMalta, before he retired thither, a second time, to assume the office of Judge.
Aseed was sown— small, indeed, in itself, and perhaps slightly covered with earth-which has since grown up and expanded into a goodly tree, bearing perennial fruit ; and wh

NOAT BLIOTHEQUE DE VILLE LA BE QUE DE LA LYON TILLE 1832 Bibliomania.]
[Bibliomania.
This curious book was first printed in 1575, and is introduced, in rather a lengthy note, at p. 349 of the Bibliomania.
349-50 of your Bibliomania.
The fact, my good Sir, is, that you have copied from the vile reprint of Laneham's letter (Warwick, 1784) , one of the most incorrect publica6 Bibliomania. ]
[Bibliomania.
"Many thanks for your acceptable present of the Bibliomania,' which being once entered on, compelled me to become a borrower of the night for a dark hour or twain ' to finish it.
Such, I maywith truth assert, has been the effect of your Bibliomania'— of which I have read as far as the note on ' CAPT.
My Harrowfriends, Messrs. DRURY, BUTLER, and EDWARDS, were also prompt in their expressions of congratulation on the appearance and certain suc- Bibliomania. ]
The latter, an experienced judge in such matters, " congratulated me on the growing ardour of the public, as well as on the credit th

Mr. Grenville's contribution was an heroic * The library of Mr. Dent (the BAROCIO of the Bibliomania and the Decameron) was sold by Mr. Evans in 1827, and produced the sum of 15,040%.
I am not sure whether we had not before exchanged notes of a common literary gossiping nature ; but in 1809, when the first edition of my Bibliomania appeared, I found myselfjustified not only in begging his acceptance of a copy of that slight performance, but in introducing him into the work itself, as one of those bibliomaniacal worthies who, assembling all the known pieces upon CHATTERTON, and collecting works upon Hawking and Rural Sports, evinced at least a book-passion which could not be considered contemptible.
In my second edition of the Bibliomania, after our acquaintance had ripened into intimacy, I introduced him into the " Auction Room" of that Bibliographical Romance, under the character of BERNARDO† ; and at this distance of time (for twenty- * See page 295.
His love of BIBLIOMANIA was stric

To someis given the power of writing well, to others the art of speaking well ; the power of subduing the soul or of captivating the heart by the magic of eloquence or of poetry.
The lack of this one object may destroy the charm of all that he has already won.
Books in these styles charm the eye, please the taste, and are the delight of their owners, but the cost of their splendid attire often exceeds their intrinsic value.
What shall we say of those libraries in which the books are all formally arranged, where the brilliant gilding and shining varnish are heightened by the ornaments of sculpture and the intermingling of exquisite vases and bronzes to give it a crowning charm ?
romances, histories of chivalry, adventures, burlesques, facetiæ, macaronic poems, treatises on magic, witchcraft and divination, memoirs of scandalous procedures, slanderous chronicles, defamatory libels and hosts of other writings inspired by an ill -regulated imagination or a cynical license.
He only is worth

Also, in perusing the sayings of the hero of this story, Theodore, we feel that Nodier sympathizes with him from the bottom of his heart.
This original and true bibliomaniac, for Nodier’s Theodore is merely a degenerate bibliophile, was celebrated at the time of the Restoration.
He handled them, asked their price, and held them up with admiring affection.
The “Bulletin du Bibliophile,” which owed its existence in part to Nodier, took upon itself the baron’s funeral oration and reproduced his will.
[2] Sacy, when he dreamed, on the eve of a book sale, of this or that volume which he coveted, and which he feared he might not be able to hold successfully against the fancy or caprice of some financier, would have palpitation of the heart.
Prévost-Paradol wrote of him: “This Christian, whom some would like to call austere, if the word austerity could cover so much forbearance and perfect gentleness, became a sort of epicure in all that concerned his reading.” The love of books was such a pa

In [ ]:
print('Done!')

Tot slot zou ik graag de concordances zien van het complete magic lexicon. Ik wist niet hoe ik ze per zin kon concordancen zoals je met het corpus van het traineeship had gedaan, maar had nu snel het zo gedaan. 

Excuses dat de code op een aantal plekken een beetje rommelig is, ik hoop dat er nog een beetje uit is op te merken wat mijn intenties waren. 